# Introduction

We tackle the problem of OCR post processing. In OCR, we map the image form of the document into the text domain. This is done first using an CNN+LSTM+CTC model, in our case based on tesseract. Since this output maps only image to text, we need something on top to validate and correct language semantics.

The idea is to build a language model, that takes the OCRed text and corrects it based on language knowledge. The langauge model could be:
- Char level: the aim is to capture the word morphology. In which case it's like a spelling correction system.
- Word level: the aim is to capture the sentence semnatics. But such systems suffer from the OOV problem.
- Fusion: to capture semantics and morphology language rules. The output has to be at char level, to avoid the OOV. However, the input can be char, word or both.

The fusion model target is to learn:

    p(char | char_context, word_context)

In this workbook we use seq2seq vanilla Keras implementation, adapted from the lstm_seq2seq example on Eng-Fra translation task. The adaptation involves:

- Adapt to spelling correction, on char level
- Pre-train on a noisy, medical sentences
- Fine tune a residual, to correct the mistakes of tesseract 
- Limit the input and output sequence lengths
- Enusre teacher forcing auto regressive model in the decoder
- Limit the padding per batch
- Learning rate schedule
- Bi-directional LSTM Encoder
- Bi-directional GRU Encoder


# Imports

In [1]:
from __future__ import print_function
import tensorflow as tf
import keras.backend as K
from keras.backend.tensorflow_backend import set_session
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Bidirectional, Concatenate, GRU
from keras import optimizers
from keras.callbacks import ModelCheckpoint, TensorBoard, LearningRateScheduler
from keras.models import load_model
import numpy as np
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from autocorrect import spell
import re
%matplotlib inline

Using TensorFlow backend.


# Utility functions

In [2]:
# Limit gpu allocation. allow_growth, or gpu_fraction
def gpu_alloc():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    set_session(tf.Session(config=config))

In [3]:
gpu_alloc()

In [4]:
def calculate_WER_sent(gt, pred):
    '''
    calculate_WER('calculating wer between two sentences', 'calculate wer between two sentences')
    '''
    gt_words = gt.lower().split(' ')
    pred_words = pred.lower().split(' ')
    d = np.zeros(((len(gt_words) + 1), (len(pred_words) + 1)), dtype=np.uint8)
    # d = d.reshape((len(gt_words)+1, len(pred_words)+1))

    # Initializing error matrix
    for i in range(len(gt_words) + 1):
        for j in range(len(pred_words) + 1):
            if i == 0:
                d[0][j] = j
            elif j == 0:
                d[i][0] = i

    # computation
    for i in range(1, len(gt_words) + 1):
        for j in range(1, len(pred_words) + 1):
            if gt_words[i - 1] == pred_words[j - 1]:
                d[i][j] = d[i - 1][j - 1]
            else:
                substitution = d[i - 1][j - 1] + 1
                insertion = d[i][j - 1] + 1
                deletion = d[i - 1][j] + 1
                d[i][j] = min(substitution, insertion, deletion)
    return d[len(gt_words)][len(pred_words)]

In [5]:
def calculate_WER(gt, pred):
    '''

    :param gt: list of sentences of the ground truth
    :param pred: list of sentences of the predictions
    both lists must have the same length
    :return: accumulated WER
    '''
#    assert len(gt) == len(pred)
    WER = 0
    nb_w = 0
    for i in range(len(gt)):
        #print(gt[i])
        #print(pred[i])
        WER += calculate_WER_sent(gt[i], pred[i])
        nb_w += len(gt[i])

    return WER / nb_w

In [6]:
def load_data_with_gt(file_name, num_samples, max_sent_len, min_sent_len, delimiter='\t', gt_index=1, prediction_index=0):
    '''Load data from txt file, with each line has: <TXT><TAB><GT>. The  target to the decoder muxt have \t as the start trigger and \n as the stop trigger.'''
    cnt = 0  
    input_texts = []
    gt_texts = []
    target_texts = []
    for row in open(file_name, encoding='utf8'):
        if cnt < num_samples :
            #print(row)
            sents = row.split(delimiter)
            input_text = sents[prediction_index]
            
            target_text = '\t' + sents[gt_index] + '\n'
            if len(input_text) > min_sent_len and len(input_text) < max_sent_len and len(target_text) > min_sent_len and len(target_text) < max_sent_len:
                cnt += 1
                
                input_texts.append(input_text)
                target_texts.append(target_text)
                gt_texts.append(sents[gt_index])
    return input_texts, target_texts, gt_texts

In [7]:
def load_data(file_name, num_samples, max_sent_len, min_sent_len):
    '''Load data from txt file, with each line has: <TXT><TAB><GT>. The  target to the decoder muxt have \t as the start trigger and \n as the stop trigger.'''
    cnt = 0  
    input_texts = []   
    
    #for row in open(file_name, encoding='utf8'):
    for row in open(file_name):
        if cnt < num_samples :            
            input_text = row           
            if len(input_text) > min_sent_len and len(input_text) < max_sent_len:
                cnt += 1                
                input_texts.append(input_text)
    return input_texts

In [8]:
def vectorize_data(input_texts, max_encoder_seq_length, num_encoder_tokens, vocab_to_int):
    
    if(len(input_texts) > max_encoder_seq_length):
        input_texts = input_texts[:max_encoder_seq_length]
    
    '''Prepares the input text and targets into the proper seq2seq numpy arrays'''
    encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length),
    dtype='float32')
    
    for i, input_text in enumerate(input_texts):
        for t, char in enumerate(input_text[:max_encoder_seq_length]):
            # c0..cn
            encoder_input_data[i, t] = vocab_to_int[char]
                
    return encoder_input_data

In [9]:
def decode_sequence(input_seq, encoder_model, decoder_model, num_decoder_tokens, max_decoder_seq_length, vocab_to_int, int_to_vocab):
    
    #print(max_decoder_seq_length)
    # Encode the input as state vectors.
    encoder_outputs, h, c  = encoder_model.predict(input_seq)
    states_value = [h,c]
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = vocab_to_int['\t']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    #print(input_seq)
    attention_density = []
    i = 0
    special_chars = ['\\', '/', '-', '—' , ':', '[', ']', ',', '.', '"', ';', '%', '~', '(', ')', '{', '}', '$', '#']
    #special_chars = []
    while not stop_condition:
        #print(target_seq)
        output_tokens, attention, h, c  = decoder_model.predict(
            [target_seq, encoder_outputs] + states_value)
        #print(attention.shape)
        attention_density.append(attention[0][0])# attention is max_sent_len x 1 since we have num_time_steps = 1 for the output
        # Sample a token
        #print(output_tokens.shape)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        
        #print(sampled_token_index)
        sampled_char = int_to_vocab[sampled_token_index]
        
        orig_char = int_to_vocab[int(input_seq[:,i][0])]
        
        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
            #print('End', sampled_char, 'Len ', len(decoded_sentence), 'Max len ', max_decoder_seq_length)
            sampled_char = ''
        
        # Copy digits as it, since the spelling corrector is not good at digit corrections
        
        if(orig_char.isdigit() or orig_char in special_chars):
            decoded_sentence += orig_char            
        else:
            if(sampled_char.isdigit() or sampled_char in special_chars):
                decoded_sentence += ''
            else:
                decoded_sentence += sampled_char
        
        #decoded_sentence += sampled_char


        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]
        
        i += 1
        if(i > 48):
            i = 0
    attention_density = np.array(attention_density)
    
    # Word level spell correct
    '''
    corrected_decoded_sentence = ''
    for w in decoded_sentence.split(' '):
        corrected_decoded_sentence += spell(w) + ' '
    decoded_sentence = corrected_decoded_sentence
    '''
    return decoded_sentence, attention_density


In [10]:
def word_spell_correct(decoded_sentence):
    if(decoded_sentence == ''):
        return ''
    corrected_decoded_sentence = ''
    special_chars = ['\\', '/', '-', '—' , ':', '[', ']', ',', '.', '"', ';', '%', '~', '(', ')', '{', '}', '$', '&', '#', '☒', '■', '☐', '□', '☑', '@']
    for w in decoded_sentence.split(' '):
        #print(w)
        if((len(re.findall(r'\d+', w))==0) and not (w in special_chars)):
            corrected_decoded_sentence += spell(w) + ' '
        else:
            corrected_decoded_sentence += w + ' '
    return corrected_decoded_sentence

In [11]:
def clean_up_sentence(sentence, vocab):
    s = ''
    prev_char = ''
    for c in sentence.strip():
        if c not in vocab or (c == ' ' and prev_char == ' '):
            s += ''
        else:
            s += c
        prev_char = c
            
    return s

# Load data

# Load model params

In [12]:
data_path = '../../dat/'

In [13]:
max_sent_lengths = [50, 100]

In [14]:
vocab_file = {}
model_file = {}
encoder_model_file = {}
decoder_model_file = {}
model = {}
encoder_model = {}
decoder_model = {}
vocab = {}
vocab_to_int = {}
int_to_vocab = {}
max_sent_len = {}
min_sent_len = {}
num_decoder_tokens = {}
num_encoder_tokens = {}
max_encoder_seq_length = {}
max_decoder_seq_length = {}

In [15]:

for i in max_sent_lengths:
    vocab_file[i] = 'vocab-{}.npz'.format(i)
    model_file[i] = 'best_model-{}.hdf5'.format(i)
    encoder_model_file[i] = 'encoder_model-{}.hdf5'.format(i)
    decoder_model_file[i] = 'decoder_model-{}.hdf5'.format(i)
    
    vocab = np.load(file=vocab_file[i])
    vocab_to_int[i] = vocab['vocab_to_int'].item()
    int_to_vocab[i] = vocab['int_to_vocab'].item()
    max_sent_len[i] = vocab['max_sent_len']
    min_sent_len[i] = vocab['min_sent_len']
    input_characters = sorted(list(vocab_to_int))
    num_decoder_tokens[i] = num_encoder_tokens[i] = len(input_characters) #int(encoder_model.layers[0].input.shape[2])
    max_encoder_seq_length[i] = max_decoder_seq_length[i] = max_sent_len[i] - 1#max([len(txt) for txt in input_texts])
    
    model[i] = load_model(model_file[i])
    encoder_model[i] = load_model(encoder_model_file[i])
    decoder_model[i] = load_model(decoder_model_file[i])

/opt/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:269: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [16]:
num_samples = 1000000
#tess_correction_data = os.path.join(data_path, 'test_data.txt')
#input_texts = load_data(tess_correction_data, num_samples, max_sent_len, min_sent_len)

OCR_data = os.path.join(data_path, 'new_trained_data.txt')
#input_texts, target_texts, gt_texts = load_data_with_gt(OCR_data, num_samples, max_sent_len, min_sent_len, delimiter='|',gt_index=0, prediction_index=1)
input_texts, target_texts, gt_texts = load_data_with_gt(OCR_data, num_samples, max_sent_len=10000, min_sent_len=0)

In [17]:
# Sample data
print(len(input_texts))
for i in range(10):
    print(input_texts[i], '\n', target_texts[i])

1951
Me dieal Provider Roles: Treating  
 	Medical Provider Roles: Treating


Provider First Name: Christine  
 	Provider First Name: Christine


Provider Last Name: Nolen, MD  
 	Provider Last Name: Nolen, MD


Address Line 1 : 7 25 American Avenue  
 	Address Line 1 : 725 American Avenue


City. W’aukesha  
 	City: Waukesha


StatefProvinee: ‘WI  
 	State/Province: WI


Postal Code: 5 31 88  
 	Postal Code: 53188


Country". US  
 	Country:  US


Business Telephone: (2 62) 92 8- 1000  
 	Business Telephone: (262) 928- 1000


Date ot‘Pirst Visit: 1 2/01f20 17  
 	Date of First Visit: 12/01/2017




In [18]:
# Spell correct before inference
'''
input_texts_ = []
for sent in input_texts:
    sent_ = ''
    for word in sent.split(' '):
        sent_ += spell(word) + ' '
    input_texts_.append(sent_)
input_texts = input_texts_
input_texts_ = []
# Sample data
print(len(input_texts))
for i in range(10):
    print(input_texts[i], '\n', target_texts[i])
'''

"\ninput_texts_ = []\nfor sent in input_texts:\n    sent_ = ''\n    for word in sent.split(' '):\n        sent_ += spell(word) + ' '\n    input_texts_.append(sent_)\ninput_texts = input_texts_\ninput_texts_ = []\n# Sample data\nprint(len(input_texts))\nfor i in range(10):\n    print(input_texts[i], '\n', target_texts[i])\n"

In [19]:
decoded_sentences = []
corrected_sentences = []

#for seq_index in range(len(input_texts)):
results = open('RESULTS.md', 'w')
results.write('|OCR sentence|GT sentence|Char decoded sentence|Word decoded sentence|Sentence length (chars)|\n')
results.write('---------------|-----------|----------------|----------------|----------------|\n')
     

for i, input_text in enumerate(input_texts):
    #print(input_text)
    # Find the input length range to choose the proper model to use
    len_range = max_sent_lengths[-1] # Take the longest range
    for length in max_sent_lengths:
        if(len(input_text) < length):
            len_range = length
            break
    #print(len_range)
    
    input_text = clean_up_sentence(input_text, vocab_to_int[len_range])
    encoder_input_data = vectorize_data(input_texts=[input_text], max_encoder_seq_length=max_encoder_seq_length[len_range], num_encoder_tokens=num_encoder_tokens[len_range], vocab_to_int=vocab_to_int[len_range])
    
    

    target_text = gt_texts[i]
    
    input_seq = encoder_input_data
    #print(input_seq.shape)
    #print(max_decoder_seq_length[len_range])
    #print(max_decoder_seq_length)
    decoded_sentence,_  = decode_sequence(input_seq, encoder_model[len_range], decoder_model[len_range], num_decoder_tokens[len_range],  max_decoder_seq_length[len_range], vocab_to_int[len_range], int_to_vocab[len_range])
    corrected_sentence = word_spell_correct(input_text)
    print('-Lenght = ', len_range)
    print('Input sentence:', input_text)
    print('GT sentence:', target_text.strip())
    print('Char Decoded sentence:', decoded_sentence)   
    print('Word Decoded sentence:', corrected_sentence) 
    results.write(' | ' + input_text + ' | ' + target_text.strip() + ' | ' + decoded_sentence + ' | ' + corrected_sentence + ' | ' + str(len_range) + ' | \n')
    decoded_sentences.append(decoded_sentence)
    corrected_sentences.append(corrected_sentence)
results.close()    

    

-Lenght =  50
Input sentence: Me dieal Provider Roles: Treating
GT sentence: Medical Provider Roles: Treating
Char Decoded sentence: Medical Provider Roles:Treating
Word Decoded sentence: Me dieal Provider Roles Treating 
-Lenght =  50
Input sentence: Provider First Name: Christine
GT sentence: Provider First Name: Christine
Char Decoded sentence: Provider First Name: Chirisin
Word Decoded sentence: Provider First Name Christine 
-Lenght =  50
Input sentence: Provider Last Name: Nolen, MD
GT sentence: Provider Last Name: Nolen, MD
Char Decoded sentence: Provider Last Name: None,Me
Word Decoded sentence: Provider Last Name Dolens MD 
-Lenght =  50
Input sentence: Address Line 1 : 7 25 American Avenue
GT sentence: Address Line 1 : 725 American Avenue
Char Decoded sentence: Address Line 1:725 nemin Avent Avent
Word Decoded sentence: Address Line 1 : 7 25 American Avenue 
-Lenght =  50
Input sentence: City. W’aukesha
GT sentence: City: Waukesha
Char Decoded sentence: City. Worances
Word De

-Lenght =  100
Input sentence: pAvMENTsm'E—I: STATEMENT DATE WILL NOT APPEAR ON THIS STATEMENT
GT sentence: PAYMENTS RECEIVED AFTER STATE DATE WILL NOT APPEAR ON THIS STATEMENT
Char Decoded sentence: ACTIVE LaU—A:SY STTAT LATE TipSU TAYATILANSTEALALDINDGTATE—
Word Decoded sentence: pAvMENTsm'E—I: STATEMENT DATE WILL NOT APPEAR ON THIS STATEMENT 
-Lenght =  50
Input sentence: DATE
GT sentence: DATE
Char Decoded sentence: DATE
Word Decoded sentence: DATE 
-Lenght =  50
Input sentence: DESCRIPTION
GT sentence: DESCRIPTION
Char Decoded sentence: DESCRIPTION
Word Decoded sentence: DESCRIPTION 
-Lenght =  50
Input sentence: _AYMENTS
GT sentence: PAYMENTS
Char Decoded sentence: PAAMENTS
Word Decoded sentence: PAYMENTS 
-Lenght =  50
Input sentence: _DJUSTMENTS
GT sentence: ADJUSTMENTS
Char Decoded sentence: ADJUT DANE
Word Decoded sentence: ADJUSTMENTS 
-Lenght =  50
Input sentence: PAT'ENT BALANCE
GT sentence: PATIENTS BALANCE
Char Decoded sentence: PATTET BAMAE
Word Decoded sentence: PATIEN

-Lenght =  100
Input sentence: To pay your bill on line with a credit card, log on to www.ebixinc.comlpayonline.html.
GT sentence: To pay your bill on line with a credit card, log on to www.ebixinc.com/payonline.html.
Char Decoded sentence: To ppedich ont old polland pollysiot onghes,ctionship pou hypent pon one ponkendent Fing o,lep
Word Decoded sentence: To pay your bill on line with a credit cards log on to www.ebixinc.comlpayonline.html. 
-Lenght =  50
Input sentence: ACCOUNT# EMA297232
GT sentence: ACCOUNT# EMA297232
Char Decoded sentence: ACCOUNT# CAE297232
Word Decoded sentence: ACCOUNT EMA297232 
-Lenght =  100
Input sentence: PLACE OF SERVICE 11 Office 21 Inpatient 22 Outpatient Hospital 23 Emergency Room-Hospital
GT sentence: PLACE OF SERVICE 11 Office 21 Inpatient 22 Outpatient Hospital 23 Emergency Room-Hospital
Char Decoded sentence: PLINent Name For11y E PAT21TANT Ficate22Sprour Ccine Fing Unum 11e MIDe M21r Cound
Word Decoded sentence: PLACE OF SERVICE 11 Office 21 Inpat

-Lenght =  100
Input sentence: To disclose information, whether from before, during or after the date of this authorization, about my health, including HIV, AIDS or other disorders of the Immune system, use of drugs or alcohol, mental or phy5ica| histor , condition, advice or treatment (except this authorization does not authorize release of psychotherapy notesi, prescription drug history, earnings, financial or credit history, professional licenses, employment history, insurance claims and benefits, and all other claims and benefits, including Social Security claims and benefits (“My Information");
GT sentence: To disclose information, whether from before, during or after the date of this authorization, about my health, including HIV, AIDS or other disorders of the immune system, use of drugs or alcohol, mental or physical history, condition, advice or treatment (except this authorization does not authorize release of psychotherapy notes, prescription drug history, earnings, financial

-Lenght =  100
Input sentence: I signed on behalf of the Insured as (Relationship). If Power of Attorney Designee, Guardian, or Conservator, please attach a copy of the document granting authority.
GT sentence: I signed on behalf of the Insured as (Relationship). If Power of Attorney Designee, Guardian, or Conservator, please attach a copy of the document granting authority.
Char Decoded sentence: I signed on behalf of the Insured as (relationshiprohent Insunt If Nomenthe fol orent( the int chenk
Word Decoded sentence: I signed on behalf of the Insured as (Relationship). If Power of Attorney Designee Guardian or Conservatory please attach a copy of the document granting authority 
-Lenght =  100
Input sentence: Unum is a registered trademark and marketing brand of Unum Group and its insuring subsidiaries.
GT sentence: Unum is a registered trademark and marketing brand of Unum Group and its insuring subsidiaries.
Char Decoded sentence: Unum paricaress rof tertstrigheristur chigstromp th

...

-Lenght =  50
Input sentence: Spou s 0 Information
GT sentence: Spouse Information
Char Decoded sentence: Spouse 0nformation
Word Decoded sentence: Spou s 0 Information 
-Lenght =  50
Input sentence: First Name:
GT sentence: First Name:
Char Decoded sentence: First Name:
Word Decoded sentence: First Name 
-Lenght =  50
Input sentence: Middle Name/initial:
GT sentence: Middle Name/Initial:
Char Decoded sentence: Middle Name/inilali:
Word Decoded sentence: Middle Name/initial: 
-Lenght =  50
Input sentence: Last Name:
GT sentence: Last Name:
Char Decoded sentence: Last Name:
Word Decoded sentence: Last Name 
-Lenght =  50
Input sentence: Social Security Number:
GT sentence: Social Security Number:
Char Decoded sentence: Social Security Number:
Word Decoded sentence: Social Security Number 
-Lenght =  50
Input sentence: Birth Date:
GT sentence: Birth Date:
Char Decoded sentence: Birth Date:
Word Decoded sentence: Birth Date 
-Lenght =  50
Input sentence: Gender:
GT sentence: Gender:
Char 

-Lenght =  50
Input sentence: Postal Code: 43623
GT sentence: Postal Code: 43623
Char Decoded sentence: Postal Code: 43623
Word Decoded sentence: Postal Code 43623 
-Lenght =  50
Input sentence: Country". US
GT sentence: Country: US
Char Decoded sentence: Country".US
Word Decoded sentence: Country US 
-Lenght =  50
Input sentence: Business Telephone: (419) 474- 1210
GT sentence: Business Telephone: (419) 474-1210
Char Decoded sentence: Business Telephone: (419) 474-1210
Word Decoded sentence: Business Telephone (419) 474- 1210 
-Lenght =  50
Input sentence: Business Fax (419) 474-3076
GT sentence: Business Fax: (419) 474-3076
Char Decoded sentence: Business Fax (419)474-3076
Word Decoded sentence: Business Fax (419) 474-3076 
-Lenght =  50
Input sentence: Date ofFirst Visit: 03/09/2018
GT sentence: Date of First Visit: 03/09/2018
Char Decoded sentence: Date of First Visi:03/09/2018
Word Decoded sentence: Date ofFirst Visit 03/09/2018 
-Lenght =  50
Input sentence: Date ofNen Visit: 03/

-Lenght =  50
Input sentence: State/PrOVirice:
GT sentence: State/Province:
Char Decoded sentence: State/Province:
Word Decoded sentence: State/PrOVirice: 
-Lenght =  50
Input sentence: Postal Code
GT sentence: Postal Code:
Char Decoded sentence: Postal Code
Word Decoded sentence: Postal Code 
-Lenght =  50
Input sentence: Country.
GT sentence: Country:
Char Decoded sentence: Country.
Word Decoded sentence: Country 
-Lenght =  50
Input sentence: Best Phone Number to be Reached During the Day:
GT sentence: Best Phone Number to be Reached During the Day:
Char Decoded sentence: Best Phone Number to be Reached During the Day:
Word Decoded sentence: Best Phone Number to be Reached During the Day 
-Lenght =  50
Input sentence: Email Address:
GT sentence: Email Address:
Char Decoded sentence: Email Address:
Word Decoded sentence: Email Address 
-Lenght =  50
Input sentence: unumQ
GT sentence: unum
Char Decoded sentence: unum
Word Decoded sentence: unumQ 
-Lenght =  50
Input sentence: The Bene

-Lenght =  100
Input sentence: I also authorize Unum to disclose My Information to the following persons (for the purpose of reporting claim status or experience, or so that the moment may carry out health care operations, claims payment, administrative or audit functions related to any benefit, plan or claim): any employee benefit plan sponsored b my employer; any person providing services or insurance benefits to (or on behalf of) my employer, any suc plan or claim, or any benefit offered by Unum; or, the Social Securit Administration. Unum will not condition the payment of insurance benefits on whether I authorize the disc osures described in this paragraph. For the purposes of these disclosures by Unum, this authorization is valid for one year or for the length of time otherwise permitted by law.
GT sentence: I also authorize Unum to disclose My Information to the following persons (for the purpose of reporting claim status or experience, or so that the recipient may carry out heal

-Lenght =  50
Input sentence: Time ofAccident: 14:45
GT sentence: Time of Accident: 14:45
Char Decoded sentence: Time of Actien:14:45
Word Decoded sentence: Time ofaccidenl 14:45 
-Lenght =  50
Input sentence: Accident Date: 02/170018
GT sentence: Accident Date: 02/17/2018
Char Decoded sentence: Accident Date: 02/17001
Word Decoded sentence: Accident Date 02/170018 
-Lenght =  50
Input sentence: Diagnosis Code: knee injury
GT sentence: Diagnosis Code: knee injury
Char Decoded sentence: Diagnosis Code: Cene injury
Word Decoded sentence: Diagnosis Code knee injury 
-Lenght =  50
Input sentence: Sn rg er)’ Information
GT sentence: Surgery Information
Char Decoded sentence: Sngrgery)Information
Word Decoded sentence: Sn rg era Information 
-Lenght =  50
Input sentence: 15 Surgery Required: No
GT sentence: Is Surgery Required: No
Char Decoded sentence: 15 Surgery Required: No
Word Decoded sentence: 15 Surgery Required No 
-Lenght =  50
Input sentence: Medical Proxitler Information , Physici

In [20]:
input_texts = ['SUBJECTIVE: This is a S-year-old +@W his left great toe with the handleh lacration.',
               'Thera was no handlebarthe lacration.',
               'Patiet last tet is needing this for school at this',
               'OBJECTIVE : The temp is 99.8, the f tha blood pressure 99/64, O2 sat 94 8/10 at this time.',
               'Left great toe the dorsl surface, extending ta th active hemorrhage at this time.',
               'Th anaathetized with a cotton ball sat Left this in place for 20 minutes.',
               'with Betadine again and injected th he tolerated very well.',
               'The wound sutures. Antibiotic eintment and g',
               'Patient tolerated very well. Pat',
               'IMPRESSION: Lacration te left grs',
               'PLAN: Patent is to do dressing ch advised as far as checking tha waurn it with soap and water.',
               'Sutures oy have any problems prior te that tim ona teaspoon three times a day rer Ibuprofan far pain, discomfort. Cg',
               'hite male who accidently dropped a bike onto ar end hitting the left great toe,'
               'causing a guard to the end of the bike, which caused anus shot is more that three years ago and time.',
               'nlse ef 105 and regular, resprations 286,% on room air.',
               'Patient rates hia pain at — there i15 noted a 3-om laceration across a lateral aspect of tha toe.',
               'There is no e toa ir cleansed with Betadine.',
               'It is then urated with 5 cu of 2% Hylocaine plain.',
               'We then cleansed ae toa with 3 cc of 2% Xylacaina plain',
               'which was then clesed with five 5-0 Prolene ressure dressing was then applied to the tos',
               'paient is given DPT 0.5 ee intramucular (IM).at toe.',
               'Kefylex 250 mg per 5 ml, the next seven days.',
               'He may use Tylenol or 11 if any problems.',
               'Unum Life Insurance Company of America 2211',               
               'Congress Street Portland, Maine 04122',
               'APPLICATION FOR GROUP CRITICAL LLNESS INSURANCE',
               'I Evidence of Insurability',
               '',
               'Application Type: @ New Enrollee Change to',
               'Existing Coverage  Reinstatement  Internal',
               'Replacement  Late Applicant  Rehire SECTION 1:',
               'Employee(Applicant) Information  Always',
               'Complete Employee Name(First, Middle, Last)',
               'Social Security Number Nikolas J Jones',
               '123 - 456 - 7890 Home Address(Street/ PO Box)',
               'Gender 1634 Stewert St  F  M City Date of Birth',
               '(mm / dd / yyyy) Seattle 06 / 15 / 1991 State Zip',
               'Code Home Phone # Washington 98101 854-555-1212',
               'Are you Actively at Work? Employee ID / Payroll #',
               ' Yes  No55624 a.Are you a U.S.Citizen or',
               'Canadian Citizen working in the U.S.? b.Are you',
               'legally authorized to work in  Yes  No(If No',
               'reply to part b) the U.S.?  Yes  No Employer',
               'Name Group Number Date of Hire(mm/ dd / yyyy)',
               'Facebook 11 - 555566 11 / 30 / 2016 Occupation',
               'Eligibility Class Software Engineer 7 Scheduled',
               'Number of Work Hours per Week Work Phone # 35',
               '854-555-6622 SECTION 2: Spouse Information ',
               'Complete Only if applying for Spouse coverage Name',
               '(First, Middle, Last) Social Security Number',
               'Gender Date of Birth(mm / dd / yyyy) Does the',
               '1019 - 07 - AZ 1',
              'if claint is for a child, please state your relationship 10 the child',
              'date of accident 3d _ time of accident ram. 0 p.m.',
              'have you slopped working? (of yes [1 no if yes, what was the last day that you worked? (mm/ddryy)_| —3 | —{% cnslamegs bil =']
               
for input_text in input_texts:
    len_range = max_sent_lengths[-1] # Take the longest range
    for length in max_sent_lengths:
        if(len(input_text) < length):
            len_range = length
            break
    #print(len_range)
    pre_corrected_sentence = word_spell_correct(input_text)
    input_text = clean_up_sentence(input_text, vocab_to_int[len_range])
    encoder_input_data = vectorize_data(input_texts=[input_text], max_encoder_seq_length=max_encoder_seq_length[len_range], num_encoder_tokens=num_encoder_tokens[len_range], vocab_to_int=vocab_to_int[len_range])



    target_text = gt_texts[i]

    input_seq = encoder_input_data
    #print(input_seq.shape)
    #print(max_decoder_seq_length[len_range])
    #print(max_decoder_seq_length)

    decoded_sentence,_  = decode_sequence(input_seq, encoder_model[len_range], decoder_model[len_range], num_decoder_tokens[len_range],  max_decoder_seq_length[len_range], vocab_to_int[len_range], int_to_vocab[len_range])
    corrected_sentence = word_spell_correct(input_text)
    #print('-Lenght = ', len_range)
    print('Input sentence:', input_text)
    #print('Spell Decoded sentence:', pre_corrected_sentence) 
    #print('Char Decoded sentence:', decoded_sentence)   
    print('Word Decoded sentence:', corrected_sentence) 
    print('\n')



Input sentence: SUBJECTIVE: This is a S-year-old +@W his left great toe with the handleh lacration.
Word Decoded sentence: SUBJECTIVE This is a S-year-old Now his left great toe with the handle laceration 


Input sentence: Thera was no handlebarthe lacration.
Word Decoded sentence: Thera was no handlebarthe laceration 


Input sentence: Patiet last tet is needing this for school at this
Word Decoded sentence: Patient last tet is needing this for school at this 


Input sentence: OBJECTIVE : The temp is 99.8, the f tha blood pressure 99/64, O2 sat 94 8/10 at this time.
Word Decoded sentence: OBJECTIVE : The temp is 99.8, the f tha blood pressure 99/64, O2 sat 94 8/10 at this time 


Input sentence: Left great toe the dorsl surface, extending ta th active hemorrhage at this time.
Word Decoded sentence: Left great toe the dorsal surface extending ta th active hemorrhage at this time 


Input sentence: Th anaathetized with a cotton ball sat Left this in place for 20 minutes.
Word Decoded 

In [21]:

input_texts = ['text',
'',
'',
'',
'',
' ',
'',
'',
'',
'Fai',
'10',
'7521509',
'(FISTDEOO)',
'at',
'11/3/2017',
'5:23:19',
'from',
'-9373834004',
'Req',
'IC',
'2017:1030525109:292E.',
'Page',
'4',
'of',
'5',
'(C)',
'',
'',
'',
' ',
'',
'',
'',
' ',
'',
'',
'',
'11/3/2017',
'FRI',
'8:26',
'FAX',
'2373834004',
'Kjooas00s',
'',
'',
'',
'as3-ursasy3',
'11:30:11',
'11/2/2017',
'vis',
'',
'',
'',
'®',
'®',
'&',
'ACCIDENT',
'CLAIM',
'FORM',
'',
'uu',
'num’',
'Tha',
'Benelits',
'Canter',
'',
'P.O.',
'Bax',
'100158,',
'Calumbin,',
'EC',
'20202-3150',
'',
'Tol-frea:',
'1-800-635-5587',
'Fax:',
'1-800-447-2488',
'',
'Gall',
'toll-free',
'Monday',
'through',
'Friday,',
'8',
'a.m.',
'lo',
'8',
'p.m,',
'Eagtarn',
'Time.',
'',
'',
'',
' ',
'',
'',
'',
' ',
'',
'',
'',
' ',
'',
'',
'',
' ',
'',
'',
'',
' ',
'',
'',
'',
' ',
'',
'',
'',
' ',
'',
'',
'',
' ',
'',
'',
'',
' ',
'',
'',
'',
'[',
'ATTENDING',
'PHYSICIAN',
'STATEMENT',
']',
'',
'',
'IneurexiPolicyt',
'alcar',
'Hama',
'(Lael',
'Name,',
'Flis!',
'Nama,',
'MI,',
'Suffix)',
'Data',
'of',
'Risth',
'{msmidrfyy)',
'-',
'',
'',
'Faupi',
'Nana',
'{Laut',
'Hume,',
'Flial',
'Numa,',
'1',
'Sut)',
'Dats',
'al',
'Bln',
'rAvad)',
'Ul',
'_',
'',
'-[ECIpENT',
'DETAILS',
']',
'',
'',
'',
' ',
'',
'',
'',
' ',
'',
'',
'',
' ',
'',
'',
'',
' ',
'',
'',
'',
'a',
'thls',
'Gundilan',
'the',
'result',
'of',
'a',
'acddental',
'inury?',
'ves',
'O',
'No',
'if',
'yas,',
'dale',
'of',
'accident',
'qre/ddlyy)',
'[1',
'0]',
'[z]e',
'[=]',
'',
'',
'',
'Is',
'Mig',
'condition',
'Lhe',
'result',
'of',
'hefer',
'employment',
'£1',
'Yes',
'pNo',
'[1',
'Unknown',
'',
'',
'',
'Plaaze',
'verily',
'treatment',
'for',
'the',
'accident',
'lalad',
'above.',
'',
'',
'',
' ',
'',
'',
'',
' ',
'',
'',
'',
' ',
'',
'',
'',
' ',
'',
'',
'',
' ',
'',
'',
'',
'Dalaw',
'of',
'Diagnosis',
'Diagncsis',
'Description',
'Prosadure',
'Procedure',
'Dascription',
'',
'Branden',
'(Including',
'|',
'Cudo',
'(GD)',
'ous',
'',
'Confinement)',
'eR',
'ap',
'HAS',
'TTT',
'',
'BEEF',
'eR',
'',
'wiz]',
'.',
'S33,5XxA',
'Hh',
'rioes',
'ey',
'race',
'Word',
'',
'awqd]',
'',
'weak',
'3',
'n',
'[aveny',
'[d',
'',
'wifi',
'Wl',
'',
'oa',
'',
'',
'',
' ',
'',
'',
'',
' ',
'',
'',
'',
' ',
'',
'',
'',
' ',
'',
'',
'',
' ',
'',
'',
'',
' ',
'',
'',
'',
' ',
'',
'',
'',
' ',
'',
'',
'',
'Has',
'lhe',
'pallet',
'bean',
'trastad',
'for',
'tha',
'same',
'ar',
'&',
'S(tilar',
'candillan',
'by',
'anolher',
'phyalelan',
'In',
'tha',
'past?',
'[1',
'Yen',
'Bho',
'',
'M',
'yor,',
'pioona',
'provid',
'tha',
'fares:',
'',
'',
'',
' ',
'',
'',
'',
'Diageosis:',
'Tramiment',
'Daten:',
'',
'',
'',
' ',
'',
'',
'',
' ',
'',
'',
'',
'id',
'ya.1',
'#dving',
'Lhe',
'patient',
'to',
'clap',
'working?',
'RECEIVED',
'',
'It',
'yes,',
'B8',
'of',
'what',
'cate?',
'(mmidkyy)',
'',
'',
'',
'[23]',
'[117]',
'',
'',
'',
'[Ih',
'cielih',
'fa',
'rotated',
'to',
'normal',
'prepnency,',
'please',
'grovida',
'tha',
'idliawing:',
'NOV',
'',
'Expecigd',
'Delivery',
'Dale',
'(mimicd/yy)',
'Aclual',
'Delivery',
'Dale',
'{mmiddlyy',
'',
'',
'',
' ',
'',
'',
'',
'Phyeiclan',
'informaiton',
'HUMAN',
'REGOURCITE',
'',
'',
'',
'FRAUD',
'NOTICE:',
'Any',
'person',
'wha',
'knowingly',
'files',
'&',
'statement',
'of',
'clalm',
'containing',
'FALSE',
'or',
'misleading',
'information',
'8',
'',
'subject',
'to',
'criminal',
'and',
'elvil',
'penallies.',
'This',
'includes',
'Attending',
'Physician',
'portions',
'of',
'the',
'claim',
'farm.',
'',
'',
'',
'CS',
'yma',
'SEAS',
'Ta',
'hve',
'glan',
'=',
'',
'The',
'above',
'statements',
'ara',
'trun',
'And',
'rompints',
'to',
'tho',
'bot',
'of',
'my',
'knowledge',
'and',
'bolluf.',
'',
'',
'',
'Physician',
'Name',
'(Lea!',
'Name,',
'Firat',
'Name,',
'MI,',
'Suita)',
'Plases',
'Print',
'Co',
'FHman',
'log',
'Mm',
'',
'/',
'‘',
'',
'',
'',
'Medical',
'Speclaty',
'[Tr',
'eactal-',
']',
'|',
'D',
'of',
'r',
'of',
'Ch',
'2',
'',
'2',
'Le',
'',
'',
'==',
'Zoi!',
'M',
'o',
'“Fanart',
'',
'',
'=',
'Balfrone',
'ie',
'2',
'Sle',
'iu',
'',
'il',
'HY',
'BY',
'1942',
'Fax',
'Number',
'yz—',
'43',
'-8',
'7775',
'Fhyalafans',
'Tax',
'ID',
'Number.',
'',
'',
'',
'Aro',
'you',
'refateq',
'to',
'hiv',
'pollen?',
'0',
'Yoe',
'LlMo',
'|',
'yes,',
'wal',
'iv',
'the',
'relelianshipT',
'',
'',
'',
' ',
'',
'',
'',
' ',
' ',
'',
'',
'',
'Physlclan',
'Slgnature',
'Date',
'',
'CL-1023',
'-2717',
'=',
'',
'',
'',
' ',
'',
'',
'',
'—',]
               
for input_text in input_texts:
    len_range = max_sent_lengths[-1] # Take the longest range
    for length in max_sent_lengths:
        if(len(input_text) < length):
            len_range = length
            break
    #print(len_range)
    pre_corrected_sentence = word_spell_correct(input_text)
    input_text = clean_up_sentence(input_text, vocab_to_int[len_range])
    encoder_input_data = vectorize_data(input_texts=[input_text], max_encoder_seq_length=max_encoder_seq_length[len_range], num_encoder_tokens=num_encoder_tokens[len_range], vocab_to_int=vocab_to_int[len_range])



    target_text = gt_texts[i]

    input_seq = encoder_input_data
    #print(input_seq.shape)
    #print(max_decoder_seq_length[len_range])
    #print(max_decoder_seq_length)

    decoded_sentence,_  = decode_sequence(input_seq, encoder_model[len_range], decoder_model[len_range], num_decoder_tokens[len_range],  max_decoder_seq_length[len_range], vocab_to_int[len_range], int_to_vocab[len_range])
    corrected_sentence = word_spell_correct(input_text)
    #print('-Lenght = ', len_range)
    #print('Input sentence:', input_text)
    #print('Spell Decoded sentence:', pre_corrected_sentence) 
    #print('Char Decoded sentence:', decoded_sentence)   
    
    #print('Word Decoded sentence:', corrected_sentence) 
    print(corrected_sentence) 
    #print('\n')



text 








Fai 
10 
7521509 
(FISTDEOO) 
at 
11/3/2017 
5:23:19 
from 
-9373834004 
Req 
IC 
2017:1030525109:292E. 
Page 
4 
of 
5 
Act 











11/3/2017 
FRI 
8:26 
FAX 
2373834004 
Kjooas00s 



as3-ursasy3 
11:30:11 
11/2/2017 
vis 



a 
a 
& 
ACCIDENT 
CLAIM 
FORM 

UU 
numb 
Tha 
Benelits 
Canter 

Poor 
Bax 
100158, 
Calumbin, 
EC 
20202-3150 

Tol-frea: 
1-800-635-5587 
Fax 
1-800-447-2488 

Gall 
toilufree 
Monday 
through 
Friday 
8 
army 
lo 
8 
pump 
Eastern 
Time 







































[ 
ATTENDING 
PHYSICIAN 
STATEMENT 
] 


IneurexiPolicyt 
altar 
Hama 
Lael 
Name 
Flisk 
Nama 
MID 
Suffix 
Data 
of 
Rieth 
{msmidrfyy) 
- 


Fault 
Nana 
Claut 
Hume 
Flail 
Numac 
1 
Suto 
Dats 
al 
Ban 
bravado 
Ul 
a 

-[ECIpENT 
DETAILS 
] 



















a 
thls 
Gundilan 
the 
result 
of 
a 
accidental 
inury 
ves 
O 
No 
if 
yas 
dale 
of 
accident 
qre/ddlyy) 
[1 
0] 
ze 
[=] 



Is 
Mig 
condition 
Lhe 
result 
of 
refer 
employment 
£1 
Yes 
pno 
[1 


In [22]:
input_texts = ['☑ @ New Enrollee ☐ Change to Existing Coverage ☐ Reinstatement']
for input_text in input_texts:
    len_range = max_sent_lengths[-1] # Take the longest range
    for length in max_sent_lengths:
        if(len(input_text) < length):
            len_range = length
            break
    #print(len_range)
    #print(input_text)
    pre_corrected_sentence = word_spell_correct(input_text)
    #print(pre_corrected_sentence)

    input_text_ = input_text
    input_text = clean_up_sentence(input_text, vocab_to_int[len_range])
    encoder_input_data = vectorize_data(input_texts=[input_text], max_encoder_seq_length=max_encoder_seq_length[len_range], num_encoder_tokens=num_encoder_tokens[len_range], vocab_to_int=vocab_to_int[len_range])



    target_text = gt_texts[i]

    input_seq = encoder_input_data
    #print(input_seq.shape)
    #print(max_decoder_seq_length[len_range])
    #print(max_decoder_seq_length)

    decoded_sentence,_  = decode_sequence(input_seq, encoder_model[len_range], decoder_model[len_range], num_decoder_tokens[len_range],  max_decoder_seq_length[len_range], vocab_to_int[len_range], int_to_vocab[len_range])
    corrected_sentence = word_spell_correct(input_text_)
    #print('-Lenght = ', len_range)
    print('Input sentence:', input_text_)
    #print('Spell Decoded sentence:', pre_corrected_sentence) 
    print('Char Decoded sentence:', decoded_sentence)   
    
    print('Word Decoded sentence:', corrected_sentence) 
    #print(corrected_sentence) 
    #print('\n')



Input sentence: ☑ @ New Enrollee ☐ Change to Existing Coverage ☐ Reinstatement
Char Decoded sentence: * Flesign th pate Une or ton EMDatientt the ton Unes Frenef
Word Decoded sentence: ☑ @ New Enrollee ☐ Change to Existing Coverage ☐ Reinstatement 


# Handwriting correction

In [30]:
num_samples = 1000000

OCR_data = os.path.join(data_path, 'handwritten_output.txt')
input_texts, target_texts, gt_texts = load_data_with_gt(OCR_data, num_samples, max_sent_len=10000, min_sent_len=0, delimiter='|', gt_index=0, prediction_index=1)

# Sample data
print(len(input_texts))
for i in range(100):
    print(input_texts[i], '\n', gt_texts[i])

3749
 is insisting on a policy of change . 
 
 is insisting on a policy of change . 
 12/29/17 
 
 12/29/17 
 SIAL)TH 
 
 SLP(L) THA 
 Arcadia CA 91007 
 
 Arcadia CA 91007 
 (012) 667 9375 
 
 (012) 6674375 
 In this 200-fathom trench the herring do not tood the botton . 
 
 In this 200-fathom trench the herring do not touch the bottom . 
 43638556X1 
 
 43638556X1 
 Pretoria 
 
 Pretoria 
 fiddaling about with bils of cost . 
 
 fiddling about with bills of cost . 
 ( Fig. 3) . Loop threed tound liite finger t 
 
 ( Fig. 3 ) . Loop thread round little finger , 
 200681383 
 
 200681383 
 for a working week of 34 to 36 houns . 
 
 for a working week of 34 to 36 hours . 
 Daugher 
 
 Daugther 
 Electronically Signed 
 
 Electronically Signed 
 15122 
 
 15122 
 50 
 
 50 
 ShE WAS MOVVD A Picvic TablE TO SWEAP leAveS And DROREd iT oN hER BSC 
 
 ShE wAs Moving A Picnic TAble To swEEp lEAvEs And DRoPEd iT on hER ToE 
 0724603309 
 
 0724603509 
 lwas car ping a box into the basement Nhe

In [24]:
#for seq_index in range(len(input_texts)):
results = open('RESULTS_HW.md', 'w')
results.write('|HW sentence|Corrected sentence|GT sentence|\n')
results.write('|---------------|-----------|----------------|\n')
decoded_sentences = []
corrected_sentences = []
for input_text, target_text in zip(input_texts, target_texts):
    len_range = max_sent_lengths[-1] # Take the longest range
    for length in max_sent_lengths:
        if(len(input_text) < length):
            len_range = length
            break
    #print(len_range)
    #print(input_text)
    pre_corrected_sentence = word_spell_correct(input_text)
    #print(pre_corrected_sentence)
    
    input_text = clean_up_sentence(input_text, vocab_to_int[len_range])
    encoder_input_data = vectorize_data(input_texts=[input_text], max_encoder_seq_length=max_encoder_seq_length[len_range], num_encoder_tokens=num_encoder_tokens[len_range], vocab_to_int=vocab_to_int[len_range])



    #target_text = target_texts[i]

    input_seq = encoder_input_data
    #print(input_seq.shape)
    #print(max_decoder_seq_length[len_range])
    #print(max_decoder_seq_length)

    decoded_sentence,_  = decode_sequence(input_seq, encoder_model[len_range], decoder_model[len_range], num_decoder_tokens[len_range],  max_decoder_seq_length[len_range], vocab_to_int[len_range], int_to_vocab[len_range])
    corrected_sentence = word_spell_correct(input_text)
    #print('-Lenght = ', len_range)
    print('Input sentence:', input_text)
    #print('Spell Decoded sentence:', pre_corrected_sentence) 
    #print('Char Decoded sentence:', decoded_sentence)   
    
    print('Word Decoded sentence:', corrected_sentence)
    print('Ground truth sentence:', target_text)
    results.write(' | ' + input_text + ' | ' + corrected_sentence + ' | '+ target_text.strip() + ' | \n')
    decoded_sentences.append(decoded_sentence)
    corrected_sentences.append(corrected_sentence)
    #print(corrected_sentence) 
    #print('\n')
    
results.close()


Input sentence: is insisting on a policy of change .
Word Decoded sentence: is insisting on a policy of change . 
Ground truth sentence: 	is insisting on a policy of change . 

Input sentence: 12/29/17
Word Decoded sentence: 12/29/17 
Ground truth sentence: 	12/29/17 

Input sentence: SIAL)TH
Word Decoded sentence: SIAL)TH 
Ground truth sentence: 	SLP(L) THA 

Input sentence: Arcadia CA 91007
Word Decoded sentence: Arcadia CA 91007 
Ground truth sentence: 	Arcadia CA 91007 

Input sentence: (012) 667 9375
Word Decoded sentence: (012) 667 9375 
Ground truth sentence: 	(012) 6674375 

Input sentence: In this 200-fathom trench the herring do not tood the botton .
Word Decoded sentence: In this 200-fathom trench the herring do not good the cotton . 
Ground truth sentence: 	In this 200-fathom trench the herring do not touch the bottom . 

Input sentence: 43638556X1
Word Decoded sentence: 43638556X1 
Ground truth sentence: 	43638556X1 

Input sentence: Pretoria
Word Decoded sentence: Pretori

Input sentence: Gabarel Adam Viles
Word Decoded sentence: Gabriel Adam Viles 
Ground truth sentence: 	Gabriel Adam VIles 

Input sentence: whiplash- S13.9X85
Word Decoded sentence: whiplash S13.9X85 
Ground truth sentence: 	whiplash - S13.4XXD 

Input sentence: 9:15
Word Decoded sentence: 9:15 
Ground truth sentence: 	9:15 

Input sentence: 3/2/2018
Word Decoded sentence: 3/2/2018 
Ground truth sentence: 	3/2/2018 

Input sentence: 0825648359
Word Decoded sentence: 0825648359 
Ground truth sentence: 	0825648359 

Input sentence: mother
Word Decoded sentence: mother 
Ground truth sentence: 	mother 

Input sentence: Robertson
Word Decoded sentence: Robertson 
Ground truth sentence: 	Robertson 

Input sentence: 045149682X2
Word Decoded sentence: 045149682X2 
Ground truth sentence: 	045149682X2 

Input sentence: Electronically Signed
Word Decoded sentence: Electronically Signed 
Ground truth sentence: 	Electronically Signed 

Input sentence: 0834540900
Word Decoded sentence: 0834540900 
Gr

Input sentence: being limited or an adjutment being made
Word Decoded sentence: being limited or an adjustment being made 
Ground truth sentence: 	being limited or an adjustment being made 

Input sentence: 2/25/18
Word Decoded sentence: 2/25/18 
Ground truth sentence: 	2/25/18 

Input sentence: Mis Nelaney of the script , and the great advantages
Word Decoded sentence: Mis Delaney of the script , and the great advantages 
Ground truth sentence: 	Miss Delaney of the script , and the great advantages 

Input sentence: 1906-09-03
Word Decoded sentence: 1906-09-03 
Ground truth sentence: 	1986-09-05 

Input sentence: 30 7 18
Word Decoded sentence: 30 7 18 
Ground truth sentence: 	30 7 18 

Input sentence: a man to mate sinple , stright- fervard thngs , and in
Word Decoded sentence: a man to mate single , stright forward things , and in 
Ground truth sentence: 	a man to make simple , straight-forward things , and in 

Input sentence: Wili be 11 German divisions in Nate
Word Decoded sentenc

Input sentence: P.O. Box 8012 Gneenstone 1616
Word Decoded sentence: Poor Box 8012 Greenstone 1616 
Ground truth sentence: 	P.O. Box 8012 Greenstone 1616 

Input sentence: DR. KeiTh Helton
Word Decoded sentence: DRY KeiTh Helton 
Ground truth sentence: 	Dr. KeiTh Helton 

Input sentence: 02/27/18
Word Decoded sentence: 02/27/18 
Ground truth sentence: 	02/27/18 

Input sentence: NtP
Word Decoded sentence: NTP 
Ground truth sentence: 	NIP 

Input sentence: 4
Word Decoded sentence: 4 
Ground truth sentence: 	4 

Input sentence: cintence Datahase 03-189
Word Decoded sentence: intense Database 03-189 
Ground truth sentence: 	Sentence Database P03-189 

Input sentence: In the Pollowing year , 1380 , the last and
Word Decoded sentence: In the Following year , 1380 , the last and 
Ground truth sentence: 	In the following year , 1380 , the last and 

Input sentence: 402-609-1500
Word Decoded sentence: 402-609-1500 
Ground truth sentence: 	402-609-1500 

Input sentence: Karnes, Jonathon L.
Word

Input sentence: WARBNER STR. At SOmRRSKT WRS
Word Decoded sentence: WARBIER STR At Somerset WAS 
Ground truth sentence: 	WARBLER STR. 21 SOMERSET WES 

Input sentence: 02-26-18
Word Decoded sentence: 02-26-18 
Ground truth sentence: 	02-26-18 

Input sentence: "
Word Decoded sentence: " 
Ground truth sentence: 	" 

Input sentence: 083 685 0052
Word Decoded sentence: 083 685 0052 
Ground truth sentence: 	083 685 0052 

Input sentence: to disaus a commoh course of action . Sir Roy is
Word Decoded sentence: to disas a common course of action . Sir Roy is 
Ground truth sentence: 	to discuss a common course of action . Sir Roy is 

Input sentence: Feb 21/2018
Word Decoded sentence: Feb 21/2018 
Ground truth sentence: 	Feb 2nd 2018 

Input sentence: 04102
Word Decoded sentence: 04102 
Ground truth sentence: 	04102 

Input sentence: M84.3A5A
Word Decoded sentence: M84.3A5A 
Ground truth sentence: 	M84.375A 

Input sentence: 0r261538
Word Decoded sentence: 0r261538 
Ground truth sentence: 	DN 

...

Input sentence: N/A
Word Decoded sentence: Na 
Ground truth sentence: 	N/A 

Input sentence: inelading a compulsory savings sheme which the
Word Decoded sentence: including a compulsory savings shame which the 
Ground truth sentence: 	including a compulsory savings scheme which the 

Input sentence: on Fetruary 20.
Word Decoded sentence: on February 20. 
Ground truth sentence: 	on February 20 . 

Input sentence: 2
Word Decoded sentence: 2 
Ground truth sentence: 	2 

Input sentence: are dealing with a noble edifice which
Word Decoded sentence: are dealing with a noble edifice which 
Ground truth sentence: 	are dealing with a noble edifice which 

Input sentence: 40122
Word Decoded sentence: 40122 
Ground truth sentence: 	40422 

Input sentence: Service to the Jxchequer will have increased over
Word Decoded sentence: Service to the Exchequer will have increased over 
Ground truth sentence: 	service to the Exchequer will have increased over 

Input sentence: 1/31/18
Word Decoded sentence

Input sentence: Cons ytoielge
Word Decoded sentence: Cons ytoielge 
Ground truth sentence: 	Chris Ethridqe 

Input sentence: 192lb.
Word Decoded sentence: 192lb. 
Ground truth sentence: 	192lb. 

Input sentence: Labour's Colonidl spokesman , maid Sir Roy had
Word Decoded sentence: labours Colonial spokesman , maid Sir Roy had 
Ground truth sentence: 	Labour's Colonial spokesman , said Sir Roy had 

Input sentence: 011023-5164
Word Decoded sentence: 011023-5164 
Ground truth sentence: 	011 023-5164 

Input sentence: 924.3
Word Decoded sentence: 924.3 
Ground truth sentence: 	924.3 

Input sentence: I
Word Decoded sentence: I 
Ground truth sentence: 	1 

Input sentence: maltind an unflincting witmess to what
Word Decoded sentence: maltine an unflinching witness to what 
Ground truth sentence: 	maintained an unflinching witness to what 

Input sentence: 513-528-1809
Word Decoded sentence: 513-528-1809 
Ground truth sentence: 	513-528-1209 

Input sentence: Pomorama
Word Decoded sentence: 

Input sentence: to Gsub-lil on the tower .
Word Decoded sentence: to Gsub-lil on the tower . 
Ground truth sentence: 	to 4sub-lil on the lower . 

Input sentence: MSonde, Anthony
Word Decoded sentence: MSonde, Anthony 
Ground truth sentence: 	McBride,Anthony 

Input sentence: 614 533 3780
Word Decoded sentence: 614 533 3780 
Ground truth sentence: 	614 533 3780 

Input sentence: Marianne, M Fischer
Word Decoded sentence: Marianne M Fischer 
Ground truth sentence: 	Marianne, M Fischer 

Input sentence: Plastic Surgery
Word Decoded sentence: Plastic Surgery 
Ground truth sentence: 	Plastic Surgery 

Input sentence: Ke
Word Decoded sentence: Ke 
Ground truth sentence: 	* 

Input sentence: 100
Word Decoded sentence: 100 
Ground truth sentence: 	100 

Input sentence: Peeting tay to slide over a 30 in. table , but the height
Word Decoded sentence: Meeting tay to slide over a 30 in table , but the height 
Ground truth sentence: 	feeding tray to slide over a 30 in. table , but the height 

Inp

Input sentence: Bl6 The met, Grand Natienal Baule vard , Miterton Ridge
Word Decoded sentence: Bl6 The met Grand National Batule hard , Miterton Ridge 
Ground truth sentence: 	B16 The Met, Grand National Boulevard , Milnerton Ridge 

Input sentence: The gauge can nou be ised to nit in the tapers
Word Decoded sentence: The gauge can not be ised to nit in the tapers 
Ground truth sentence: 	The gauge can now be used to nick in the tapers 

Input sentence: 27-1217
Word Decoded sentence: 27-1217 
Ground truth sentence: 	27.12.47 

Input sentence: as above
Word Decoded sentence: as above 
Ground truth sentence: 	as above 

Input sentence: laceration repain
Word Decoded sentence: laceration remain 
Ground truth sentence: 	laceration repair 

Input sentence: OUrpt 2/16/18
Word Decoded sentence: Out 2/16/18 
Ground truth sentence: 	OUTpt 2/16/18 

Input sentence: EOAAaR N8 3 . He rulftled the stirels irritably and
Word Decoded sentence: EOAAaR N8 3 . He ruffled the stires irritably and 
Ground

Input sentence: Maigaietha Sisama Jchaia Potgieter.
Word Decoded sentence: Maigaietha Sama Achaia Potgieter. 
Ground truth sentence: 	Margaretha Susanna Johanna Potgieter 

Input sentence: Lowdermilk, Charity
Word Decoded sentence: Lowdermilk Charity 
Ground truth sentence: 	Lowdermilk, Charity 

Input sentence: 01/08/2018
Word Decoded sentence: 01/08/2018 
Ground truth sentence: 	01/08/2018 

Input sentence: 3-14-18
Word Decoded sentence: 3-14-18 
Ground truth sentence: 	3-14-18 

Input sentence: tent of the paper , and in
Word Decoded sentence: tent of the paper , and in 
Ground truth sentence: 	tent of the taper , and in 

Input sentence: script , and the great adventages to be
Word Decoded sentence: script , and the great advantages to be 
Ground truth sentence: 	script , and the great advantages to be 

Input sentence: Lane
Word Decoded sentence: Lane 
Ground truth sentence: 	Lane 

Input sentence: 044827038X9
Word Decoded sentence: 044827038X9 
Ground truth sentence: 	044827038X9

In [31]:
WER_spell_correction = calculate_WER(gt_texts, decoded_sentences)
print('WER_spell_correction |TEST= ', WER_spell_correction)

WER_spell_correction |TEST=  0.200670591446


In [32]:
WER_spell_word_correction = calculate_WER(gt_texts, corrected_sentences)
print('WER_spell_word_correction |TEST= ', WER_spell_word_correction)

WER_spell_word_correction |TEST=  0.0458757326471


In [33]:
WER_OCR = calculate_WER(gt_texts, input_texts)
print('WER_OCR |TEST= ', WER_OCR)

WER_OCR |TEST=  0.145528740562
